# 15장. 텐서보드(TensorBoard) 활용

In [ ]:
!taskkill /IM "tensorboard.exe" /F

In [ ]:
!rmdir /s /q logs

In [ ]:
%load_ext tensorboard

In [ ]:
from datetime import datetime
import tensorflow as tf
import numpy as np

In [ ]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()

## 15.1 그래프 그리기

In [ ]:
domain = np.linspace(-1, 1)
for s, x in enumerate(domain):
    y = np.cos(2 * np.pi * x)
    tf.summary.scalar("Cosine", data=y, step=s)

## 15.2 히스토그램 그리기

In [ ]:
N = 400
for k in range(N):
    mu = k / float(N)
    val = tf.random.normal(shape=[1000], mean=mu, stddev=1)
    tf.summary.histogram("normal/moving_mean", val, k)

## 15.3 이미지 그리기

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
N = 400
for k in range(N):
    anImage = np.expand_dims(x_train[k], axis=[0, -1])
    tf.summary.image("MNIST Image", anImage, k)

In [ ]:
%tensorboard --logdir logs/scalars --port=3982

## 15.4 신경망 모델 학습 과정에 텐서보드 적용하기

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
logdir = "logs/fit1/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.SGD(learning_rate=0.2),
    metrics=['accuracy']
)

training_history = model.fit(x_train, y_train, epochs=75,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback],
)

print("Average test loss: ", np.average(training_history.history['loss']))

In [ ]:
%tensorboard --logdir logs/fit1 --port=3983

## 15.5 Custom 값을 텐서보드에 출력하기

In [ ]:
from tensorflow import keras

logdir = "logs/fit2/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()

def lr_schedule(epoch):
    learning_rate = 0.2
    if epoch > 10:
        learning_rate = 0.02
    if epoch > 20:
        learning_rate = 0.01
    if epoch > 50:
        learning_rate = 0.005

    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

lr_callback = keras.callbacks.LearningRateScheduler(lr_schedule)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.SGD(),
    metrics=['accuracy']
)

training_history = model.fit(x_train, y_train, epochs=75,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback, lr_callback],
)

print("Average test loss: ", np.average(training_history.history['loss']))

In [ ]:
%tensorboard --logdir logs/fit2  --port=3984